In [14]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt

# 1. 장치 설정 (GPU 사용 가능 시 GPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 데이터 전처리 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # MobileNetV2 입력 크기
    transforms.ToTensor()
])

# 3. 데이터셋 로드 (ImageFolder 사용)
trainset_path = 'C:/Users/jongcheol/OneDrive/바탕 화면/Semester2/train_data'
dataset = ImageFolder(root=trainset_path, transform=transform)
labels = np.array([label for _, label in dataset.imgs])  # StratifiedKFold에 사용할 레이블 배열

# 4. K-Fold Cross Validation 설정
k = 5
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
fold_accuracies = []  # 각 폴드의 검증 정확도를 저장할 리스트

# 5. 학습 과정 시각화를 위한 리스트 초기화
train_losses = []  # 에포크별 학습 손실 저장
val_accuracies = []  # 각 폴드별 검증 정확도 저장

# 6. Cross Validation 학습 및 평가
for fold, (train_idx, val_idx) in enumerate(kf.split(np.zeros(len(labels)), labels)):
    print(f"\n=== Fold {fold + 1} 시작 ===")

    # Fold별 데이터셋 분리
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=0)

    # MobileNetV2 모델 설정 (사전 학습되지 않은 가중치 사용)
    model = models.mobilenet_v2(pretrained=False)
    num_classes = len(dataset.classes)  # 데이터셋의 클래스 수
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    model = model.to(device)

    # 손실 함수 및 옵티마이저 설정
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # MobileNetV2 학습
    num_epochs = 30  # 예시로 10 에포크 학습
    fold_train_losses = []  # 현재 폴드의 에포크별 손실 저장
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        fold_train_losses.append(epoch_loss)  # 에포크 손실 저장
        print(f"Fold [{fold + 1}], Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    # 현재 폴드의 학습 손실 저장
    train_losses.append(fold_train_losses)

    # 모델 성능 평가
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.numpy())

    # Fold 검증 정확도 계산 및 저장
    fold_accuracy = accuracy_score(val_labels, val_preds)
    fold_accuracies.append(fold_accuracy)
    val_accuracies.append(fold_accuracy * 100)  # 각 폴드의 검증 정확도 저장
    print(f"Fold {fold + 1} Accuracy: {fold_accuracy * 100:.2f}%")

# 최종 K-Fold 평균 정확도 출력
print("\n=== 최종 K-Fold 평균 정확도 ===")
print(f"Average {k}-Fold Accuracy: {np.mean(fold_accuracies) * 100:.2f}%")

# 학습 과정 시각화
# 1. 학습 손실 시각화
plt.figure(figsize=(12, 5))
for i, fold_losses in enumerate(train_losses, 1):
    plt.plot(fold_losses, label=f'Fold {i} Train Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss per Epoch")
plt.legend()
plt.show()

# 2. 검증 정확도 시각화
plt.figure(figsize=(8, 5))
plt.plot(range(1, k + 1), val_accuracies, marker='o', linestyle='-', color='b')
plt.xlabel("Fold")
plt.ylabel("Validation Accuracy (%)")
plt.title(f"{k}-Fold Validation Accuracy")
plt.grid()
plt.show()




=== Fold 1 시작 ===


c:\Users\jongcheol\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jongcheol\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Fold [1], Epoch [1/30], Loss: 1.8716
Fold [1], Epoch [2/30], Loss: 1.5361
Fold [1], Epoch [3/30], Loss: 1.4280
Fold [1], Epoch [4/30], Loss: 1.3108
Fold [1], Epoch [5/30], Loss: 1.2191
Fold [1], Epoch [6/30], Loss: 1.1247
Fold [1], Epoch [7/30], Loss: 1.0517
Fold [1], Epoch [8/30], Loss: 0.9834
Fold [1], Epoch [9/30], Loss: 0.8873
Fold [1], Epoch [10/30], Loss: 0.7920
Fold [1], Epoch [11/30], Loss: 0.7106
Fold [1], Epoch [12/30], Loss: 0.6378
Fold [1], Epoch [13/30], Loss: 0.5748
Fold [1], Epoch [14/30], Loss: 0.5417
Fold [1], Epoch [15/30], Loss: 0.4456
Fold [1], Epoch [16/30], Loss: 0.4258
Fold [1], Epoch [17/30], Loss: 0.2806
Fold [1], Epoch [18/30], Loss: 0.3302
Fold [1], Epoch [19/30], Loss: 0.2467
Fold [1], Epoch [20/30], Loss: 0.2064
Fold [1], Epoch [21/30], Loss: 0.2116
Fold [1], Epoch [22/30], Loss: 0.2222
Fold [1], Epoch [23/30], Loss: 0.2474
Fold [1], Epoch [24/30], Loss: 0.1881
Fold [1], Epoch [25/30], Loss: 0.1886
Fold [1], Epoch [26/30], Loss: 0.1426
Fold [1], Epoch [27/3

c:\Users\jongcheol\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jongcheol\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


KeyboardInterrupt: 